# Modified Multiplication Table Method

## Definir fonction

fonction objective

In [40]:
import sympy as sp
import numpy as np
import math

from networkx.classes import is_empty
from numpy.matlib import empty


global variables
variables=[]
dictionaire_auxiliary_variable={}
res=1

def variable_name(n,prefix):  # np est len de p en binaire, nq est len de q en binaire
    var_name_list = []
    for i in range(n):
        var_name_list.append(prefix+str(i+1))
    return var_name_list

def create_variables(var_name):
    if isinstance(var_name, str) :
        try:
            symbol = sp.symbols(var_name)
            globals()[var_name] = symbol
            variables.append(symbol)
        except ValueError:
            print(f"{var_name} ValueError")
    return symbol

def create_varibales_from_list(list):
    for element in list:
        create_variables(element)


In [41]:
p_varname = variable_name(4,'p')
q_varname = variable_name(4,'q')
c_varname = variable_name(4,'c')

create_varibales_from_list(p_varname)
create_varibales_from_list(q_varname)
create_varibales_from_list(c_varname)

In [42]:
expr = (
    (2 * sp.Symbol('p2') + 2 * sp.Symbol('p1') * sp.Symbol('q1') + 2 * sp.Symbol('q2')
     - 8 * sp.Symbol('c2') - 4 * sp.Symbol('c1') + sp.Symbol('p1') + sp.Symbol('q1') - 3)**2
    + (2 * sp.Symbol('q1') + 2 * sp.Symbol('p2') * sp.Symbol('q2') + 2 * sp.Symbol('p1')
       + 2 * sp.Symbol('c2') - 8 * sp.Symbol('c4') - 4 * sp.Symbol('c3')
       + sp.Symbol('p2') * sp.Symbol('q1') + sp.Symbol('p1') * sp.Symbol('q2')
       + sp.Symbol('c1') + 1)**2
    + (sp.Symbol('q2') + sp.Symbol('p2') + sp.Symbol('c3') + 2 * sp.Symbol('c4') - 2)**2
)

expr

(c3 + 2*c4 + p2 + q2 - 2)**2 + (-4*c1 - 8*c2 + 2*p1*q1 + p1 + 2*p2 + q1 + 2*q2 - 3)**2 + (c1 + 2*c2 - 4*c3 - 8*c4 + p1*q2 + 2*p1 + p2*q1 + 2*p2*q2 + 2*q1 + 1)**2

In [43]:
def reduct_variable_power_term(term):
    new_term = 1
    for variable,power in term.as_powers_dict().items():
        new_term *= variable
    return new_term

def reduct_variable_power_function(expr):
    expanded_expr = sp.expand(expr)
    expanded_reducted = 0
    for term in expanded_expr.args:
        new_term = reduct_variable_power_term(term)
        expanded_reducted += new_term
    return expanded_reducted

# les varaiables sont sort par la lettre puis un nombre: "x12" -> "x" 12
def sort_key(variable):
    name = str(variable)
    # Split le nom du variable aux deux parties: letter & number
    letter = name[0]
    number = int(name[1:])
    return (letter, number)

def rest_term(coef , rest_variable_liste):
    new_term = coef
    while(len(rest_variable_liste)!=0):
        new_term = new_term * rest_variable_liste[0]
        rest_variable_liste.pop()
    return new_term

# si #variables > 2 : diemension -1
def reduct_demension_term(coef,variables_liste):
    global res

    # sélection les 3 premières variables
    [x_1, x_2, x_3] = variables_liste[:3]

    # garder le reste des variables
    rest_variables_liste= variables_liste[3:]
    rest_expr = rest_term(coef , rest_variables_liste)

    # commence la réduction dimension
    tup=(x_1,x_2)
    #check dict s'il existe déjà une variable auxiliary qui = les deux meme variable
    key = next((k for k, v in dictionaire_auxiliary_variable.items() if v == tup), None)
    if key==None:#sinon, il faut creer une variable auxiliary
        var_name = "x" + str(res)
        x_4 = create_variables(var_name)
        dictionaire_auxiliary_variable[x_4]=(x_1,x_2)
        res += 1
    else:
        x_4 = key

    expr_reduct_formula = x_4 * x_3 + 2 * (x_1 * x_2 - 2 * x_1 * x_4 - 2 * x_2 * x_4 + 3 * x_4)

    expr_reduct = rest_expr * expr_reduct_formula
    return sp.expand(expr_reduct)

def reduct_demension_expression(expr):
    new_expr = 0
    need_reduction = False

    for term in expr.args:
        # get coefficant(la constante) et variables(dans une liste)
        variables_liste = sorted(term.free_symbols, key=sort_key)
        coef = term.as_coeff_mul()[0]

        # pour les termes qui ont plus de 2 variables, il faut appliquer la formule de réduction
        if len(variables_liste) > 2 :
            if len(variables_liste) > 3 :
                need_reduction = True
            new_term = reduct_demension_term(coef, variables_liste)
            new_expr +=  new_term
        else:
            # sinon, on ne fait rien et les met directement dans la nouvelle expression
            new_expr += term

    if need_reduction: # récursive
        return reduct_demension_expression(new_expr)
    else:
        return new_expr

In [44]:
expr_reduct_power = reduct_variable_power_function(expr)
reduct_dimension_expr = reduct_demension_expression(expr_reduct_power)
reduct_dimension_expr

68*c1*c2 - 8*c1*c3 - 16*c1*c4 - 32*c1*p1 - 4*c1*p2 - 4*c1*q1 - 16*c1*q2 + 56*c1*x3 - 24*c1*x7 + 43*c1 - 16*c2*c3 - 32*c2*c4 - 64*c2*p1 - 8*c2*p2 - 8*c2*q1 - 32*c2*q2 + 112*c2*x1 - 48*c2*x8 + 120*c2 + 68*c3*c4 - 32*c3*p1 - 46*c3*p2 - 16*c3*q1 + 2*c3*q2 + 96*c3*x4 + 32*c3*x6 + 5*c3 - 64*c4*p1 - 92*c4*p2 - 32*c4*q1 + 4*c4*q2 + 192*c4*x2 + 64*c4*x5 + 44*c4 + 60*p1*p2 + 34*p1*q1 - 5*p1*q2 + 112*p1*x1 - 48*p1*x10 + 32*p1*x12 + 56*p1*x3 + 64*p1*x5 + 32*p1*x6 - 112*p1*x9 + 3*p1 + 35*p2*q1 + 2*p2*q2 - 48*p2*x11 + 32*p2*x13 + 192*p2*x2 + 96*p2*x4 - 24*p2*x7 - 48*p2*x8 - 112*p2*x9 - 11*p2 + 8*q1*q2 - 32*q1*x1 - 48*q1*x10 - 48*q1*x11 - 8*q1*x14 - 16*q1*x2 - 16*q1*x3 - 8*q1*x4 + 2*q1*x7 + 4*q1*x8 + 12*q1*x9 + 3*q1 + 4*q2*x1 + 12*q2*x10 + 12*q2*x11 + 32*q2*x12 + 32*q2*x13 - 8*q2*x14 - 32*q2*x2 + 2*q2*x3 - 16*q2*x4 - 16*q2*x5 - 8*q2*x6 + 4*q2*x7 + 8*q2*x8 + 28*q2*x9 - 11*q2 - 168*x1 + 72*x10 + 72*x11 - 8*x12*x9 - 48*x12 - 8*x13*x9 - 48*x13 + 2*x14*x9 + 12*x14 - 288*x2 - 84*x3 - 144*x4 - 96*x5 - 48*x6

In [45]:
print(type(reduct_dimension_expr))

<class 'sympy.core.add.Add'>


In [46]:
def transfers_qubo2Ising(reduct_demension_expr):
    subs_relations = {(variable, ( variable + 1 ) /2) for variable in variables}

    # Après la substitution, variables dans l'expression sont des variables d'Ising dans {-1,+1}
    new_expr = reduct_demension_expr.subs(subs_relations)
    new_expr = sp.expand(new_expr)
    return new_expr

expr_ising = transfers_qubo2Ising(reduct_dimension_expr)

import dimod
from dwave.preprocessing.composites import SpinReversalTransformComposite

def get_solver_parameter(expr_ising):
    linear = {}
    quadratic ={}
    offset = 0

    for term in expr_ising.args:
        variables_liste = sorted(term.free_symbols, key=sort_key)
        coef = term.as_coeff_mul()[0]
        if len(term.free_symbols) ==0:
            offset = coef
        if len(term.free_symbols) ==1:
            linear[variables_liste[0].name] = coef
        elif len(term.free_symbols) ==2:
            quadratic[(variables_liste[0].name,variables_liste[1].name)] = coef

    return (linear,quadratic,offset)

(linear,quadratic,offset) = get_solver_parameter(expr_ising)

In [47]:
print(linear)

{'x2': -60, 'x1': -35, 'x4': -30, 'x5': -20, 'x12': -10, 'x13': -10, 'x6': -10, 'x10': 15, 'x11': 15, 'x8': 15, 'c4': 45, 'c2': 53, 'q1': -135/4, 'x3': -35/2, 'q2': -5/4, 'x14': 5/2, 'p2': 13/4, 'x7': 15/2, 'p1': 39/4, 'c3': 45/2, 'c1': 53/2, 'x9': 69/2}


In [48]:
quadratic

{('c4', 'q2'): 1,
 ('q1', 'x8'): 1,
 ('q2', 'x1'): 1,
 ('q2', 'x7'): 1,
 ('c3', 'q2'): 1/2,
 ('p2', 'q2'): 1/2,
 ('q1', 'x7'): 1/2,
 ('q2', 'x3'): 1/2,
 ('x9', 'x14'): 1/2,
 ('c1', 'p2'): -1,
 ('c1', 'q1'): -1,
 ('p1', 'x9'): -28,
 ('p2', 'x9'): -28,
 ('c4', 'p2'): -23,
 ('c2', 'p1'): -16,
 ('c4', 'p1'): -16,
 ('c2', 'x8'): -12,
 ('p1', 'x10'): -12,
 ('p2', 'x11'): -12,
 ('p2', 'x8'): -12,
 ('q1', 'x10'): -12,
 ('q1', 'x11'): -12,
 ('c1', 'p1'): -8,
 ('c2', 'c4'): -8,
 ('c2', 'q2'): -8,
 ('c3', 'p1'): -8,
 ('c4', 'q1'): -8,
 ('q1', 'x1'): -8,
 ('q2', 'x2'): -8,
 ('c1', 'x7'): -6,
 ('p2', 'x7'): -6,
 ('c1', 'c4'): -4,
 ('c1', 'q2'): -4,
 ('c2', 'c3'): -4,
 ('c3', 'q1'): -4,
 ('q1', 'x2'): -4,
 ('q1', 'x3'): -4,
 ('q2', 'x4'): -4,
 ('q2', 'x5'): -4,
 ('c1', 'c3'): -2,
 ('c2', 'p2'): -2,
 ('c2', 'q1'): -2,
 ('q1', 'x14'): -2,
 ('q1', 'x4'): -2,
 ('q2', 'x14'): -2,
 ('q2', 'x6'): -2,
 ('x9', 'x12'): -2,
 ('x9', 'x13'): -2,
 ('q1', 'q2'): 2,
 ('q2', 'x8'): 2,
 ('q1', 'x9'): 3,
 ('q2', 'x10'

In [49]:
import numpy as np

def dict_to_matrix(data_dict):
    # 收集所有唯一的节点
    nodes = set()
    for key in data_dict.keys():
        nodes.add(key[0])
        nodes.add(key[1])

    # 对节点进行排序，确保顺序固定
    sorted_nodes = sorted(nodes)

    # 创建节点到索引的映射
    node_to_index = {node: i for i, node in enumerate(sorted_nodes)}

    # 创建零矩阵
    matrix_size = len(sorted_nodes)
    matrix = np.zeros((matrix_size, matrix_size))

    # 填充矩阵
    for (node1, node2), value in data_dict.items():
        i = node_to_index[node1]
        j = node_to_index[node2]
        matrix[i, j] = value

    return matrix, sorted_nodes

In [50]:
matrix, nodes = dict_to_matrix(quadratic)
print(matrix)

[[  0.    17.    -2.    -4.    -8.    -1.    -1.    -4.     0.     0.
    0.     0.     0.     0.     0.    14.     0.     0.     0.    -6.
    0.     0.  ]
 [  0.     0.    -4.    -8.   -16.    -2.    -2.    -8.    28.     0.
    0.     0.     0.     0.     0.     0.     0.     0.     0.     0.
  -12.     0.  ]
 [  0.     0.     0.    17.    -8.   -11.5   -4.     0.5    0.     0.
    0.     0.     0.     0.     0.     0.    24.     0.     8.     0.
    0.     0.  ]
 [  0.     0.     0.     0.   -16.   -23.    -8.     1.     0.     0.
    0.     0.     0.     0.    48.     0.     0.    16.     0.     0.
    0.     0.  ]
 [  0.     0.     0.     0.     0.    15.     8.5   -1.25  28.   -12.
    0.     8.     0.     0.     0.    14.     0.    16.     8.     0.
    0.   -28.  ]
 [  0.     0.     0.     0.     0.     0.     8.75   0.5    0.     0.
  -12.     0.     8.     0.    48.     0.    24.     0.     0.    -6.
  -12.   -28.  ]
 [  0.     0.     0.     0.     0.     0.     0.     2.   

# action

Exemple: N= 143 = 13 * 11

In [51]:
# Créer un symbole à partir d'un nom en string
def create_variables(var_name):
    if isinstance(var_name, str) :
        try:
            symbol = sp.symbols(var_name)
            globals()[var_name] = symbol
            variables.append(symbol)
        except ValueError:
            print(f"{var_name} ValueError")
    return symbol

def set_variable(l,prefix):
    # length de p et q sont suposés logN/2
    #length_N = math.ceil(math.log2(N))
    #l1 = l2 = math.ceil(length_N/2)

    expr = 1
    var_binary = [1]
    for i in range(1,l-1):
        var_name = (prefix+ str(i))
        var = create_variables(var_name)
        expr = 2**i * var + expr
        var_binary.append(var)
    expr = expr + 2 ** (l-1)
    var_binary.append(1)
    var_binary = var_binary[::-1]
    return (expr,var_binary)

def get_N_binary(N):
    binary_str = np.binary_repr(N)
    binary_list = [int(bit) for bit in binary_str]
    return binary_list

In [52]:
(p_expr,p_binary) = set_variable(4,"p")
print(f"p_expr = {p_expr}")
print(f"p_binary = {p_binary}")

(q_expr,q_binary) = set_variable(4,"q")
print(f"q_expr = {q_expr}")
print(f"q_binary = {q_binary}")

N_binary = get_N_binary(143)
print(f"N_binary = {N_binary}")

p_expr = 2*p1 + 4*p2 + 9
p_binary = [1, p2, p1, 1]
q_expr = 2*q1 + 4*q2 + 9
q_binary = [1, q2, q1, 1]
N_binary = [1, 0, 0, 0, 1, 1, 1, 1]


## table de mulp

ligne par ligne

In [53]:
def table_multiplication_ligne(coef,p_binary):
    lst = [coef * element for element in p_binary]
    return lst

def table_multiplication(p_binary,q_binary,N_binary):
    table = []
    lst = q_binary.copy()
    res = 0
    while not lst == []:
        element = lst.pop()
        ele = table_multiplication_ligne(element,p_binary)
        for i in range(res):
            ele.append(0)
        table.append(ele)
        res+=1
    table.append(N_binary)
    return table


In [54]:
table = table_multiplication(p_binary,q_binary,N_binary)
table

[[1, p2, p1, 1],
 [q1, p2*q1, p1*q1, q1, 0],
 [q2, p2*q2, p1*q2, q2, 0, 0],
 [1, p2, p1, 1, 0, 0, 0],
 [1, 0, 0, 0, 1, 1, 1, 1]]

In [55]:
def adjust_table(table):
    max_cols = max(len(row) for row in table)
    complet_table = np.array([[0]*(max_cols - len(row)) + row for row in table], dtype=object)
    return complet_table

In [56]:
table_complet = adjust_table(table)
print(table_complet)

[[0 0 0 0 1 p2 p1 1]
 [0 0 0 q1 p2*q1 p1*q1 q1 0]
 [0 0 q2 p2*q2 p1*q2 q2 0 0]
 [0 1 p2 p1 1 0 0 0]
 [1 0 0 0 1 1 1 1]]


## explore la table par bloc

In [89]:
col_per_bloc = 2
rows,columns = table_complet.shape
print(rows,columns)
col_1 = table_complet[:, columns - 2 ]
col_2 = table_complet[:, columns - 3 ]
print(col_1,col_2)

c_res = 1 #c_i

5 8
[p1 q1 0 0 1] [p2 p1*q1 q2 0 1]


In [110]:
def num_carry(col):
    expr = sum(col[:-1])
    max = len(expr.args) # suppose all the terms have value 1 so that the number of the terms is the max value of the sum of the column
    binary_str = np.binary_repr(max)
    last_bit = binary_str[-1]

    if last_bit != col[-1]:
        max -= 1
    num = len(np.binary_repr(max)) - 1
    return num

In [111]:
print(num_carry(col_1))
print(num_carry(col_2))

0
1


In [ ]:
def define_carry_var(num_carry):
    for i in range(num_carry):
        create_variables("c" + str(c_res))

In [ ]:
def define_function_bloc(col_1,col_2):

